In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imutils
import os
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
pip install xmltodict

In [ ]:
import xmltodict

In [ ]:
cd drive/MyDrive/CSC474/Project

/content/drive/MyDrive/CSC474/Project


In [ ]:
images = []
annotations = []

image_dir = os.getcwd() + '/data/images'
annot_dir = os.getcwd() + '/data/annotations'

for filename in os.listdir(annot_dir):
    annotations.append(filename)

for filename in os.listdir(image_dir):
    images.append(filename)

images = sorted(images)
annotations = sorted(annotations)

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

bboxes = []
dataimage = []
classes = {'without_mask':0, 'with_mask':1, 'mask_weared_incorrect':2}

for i in range (0,len(annotations)):
    annot_file = open(os.path.join(annot_dir, annotations[i]), 'r')
    image_filename = os.path.join(image_dir, images[i])

    annot_data = annot_file.read()
    annot_dict = xmltodict.parse(annot_data)['annotation']

    image = Image.open(image_filename)

    objects = annot_dict['object']
    if isinstance(objects, list):
        for item in objects:
          xmin = int(item['bndbox']['xmin'])
          ymin = int(item['bndbox']['ymin'])
          xmax = int(item['bndbox']['xmax'])
          ymax = int(item['bndbox']['ymax'])
          label = classes[item['name']] # 1-hot encoding.

          bbox = (xmin, ymin, xmax, ymax, label)
          bboxes.append(bbox)
  
          image = image.resize((224,224), Image.ANTIALIAS)
          image = image.convert(mode='RGB')
          image_arr = img_to_array(image)
          dataimage.append(image_arr)

    else:
        xmin = int(item['bndbox']['xmin'])
        ymin = int(item['bndbox']['ymin'])
        xmax = int(item['bndbox']['xmax'])
        ymax = int(item['bndbox']['ymax'])
        label = classes[item['name']] # 1-hot encoding.

        bbox = (xmin, ymin, xmax, ymax, label)
        bboxes.append(bbox)
  
        image = image.resize((224,224), Image.ANTIALIAS)
        image = image.convert(mode='RGB')
        image_arr = img_to_array(image)
        dataimage.append(image_arr)

In [ ]:
x = torch.from_numpy(np.array(dataimage))
x = torch.unsqueeze(x, -1)
x = x[0].reshape((1,3,224,224))

In [ ]:
config = [
    (32, 3, 1),
    (64, 3, 2),
    ["B", 1],
    (128, 3, 2),
    ["B", 2],
    (256, 3, 2),
    ["B", 8],
    (512, 3, 2),
    ["B", 8],
    (1024, 3, 2),
    ["B", 4],  # To this point is Darknet-53
    (512, 1, 1),
    (1024, 3, 1),
    "S",
    (256, 1, 1),
    "U",
    (256, 1, 1),
    (512, 3, 1),
    "S",
    (128, 1, 1),
    "U",
    (128, 1, 1),
    (256, 3, 1),
    "S",
]

In [ ]:
# Hyperparameters
leakyReLuAlpha = 0.1
bn_momentum = 0.999
conv_bias = False

In [40]:
class yolo_model(nn.Module):

    def __init__(self, leakyReLuAlpha = 0.1, bn_momentum = 0.999, usebias = False):
        super(yolo_model, self).__init__()
        self.leakyReLuAlpha = leakyReLuAlpha
        self.bn_momentum = bn_momentum
        self.usebias = usebias
        self.layers = self.createModelLayers()

    def createModelLayers(self):
        layers = nn.ModuleList()

        ''' convs '''
        layer = nn.Conv2d(3, 32, 3, 1, bias= self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(32, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        layer = nn.Conv2d(32, 64, 3, 2, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(64, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' residual '''
        layer1a = nn.Conv2d(64, 32, 1, bias = self.usebias, padding=0)
        layer1b = nn.BatchNorm2d(32, momentum= self.bn_momentum)
        layer1c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)

        layer2a = nn.Conv2d(32, 64, 3, bias= self.usebias, padding=1)
        layer2b = nn.BatchNorm2d(64, momentum= self.bn_momentum)
        layer2c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        
        layer = nn.Sequential(layer1a,layer1b,layer1c,layer2a,layer2b,layer2c)
        layers.append(layer)
        
        ''' convs '''
        layer = nn.Conv2d(64, 128, 3, 2, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(128, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' 2 resiudal repeats '''
        layer1a = nn.Conv2d(128, 64, 1, bias = self.usebias, padding=0)
        layer1b = nn.BatchNorm2d(64, momentum= self.bn_momentum)
        layer1c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)

        layer2a = nn.Conv2d(64, 128, 3, bias= self.usebias, padding=1)
        layer2b = nn.BatchNorm2d(128, momentum= self.bn_momentum)
        layer2c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        
        for i in range(0,2):
          layer = nn.Sequential(layer1a,layer1b,layer1c,layer2a,layer2b,layer2c)
          layers.append(layer)

        ''' convs '''
        layer = nn.Conv2d(128, 256, 3, 2, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(256, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' 8 residual repeats '''
        layer1a = nn.Conv2d(256, 128, 1, bias = self.usebias, padding=0)
        layer1b = nn.BatchNorm2d(128, momentum= self.bn_momentum)
        layer1c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)

        layer2a = nn.Conv2d(128, 256, 3, bias= self.usebias, padding=1)
        layer2b = nn.BatchNorm2d(256, momentum= self.bn_momentum)
        layer2c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        
        for i in range(0,8):
          layer = nn.Sequential(layer1a,layer1b,layer1c,layer2a,layer2b,layer2c)
          layers.append(layer)

        ''' convs '''
        layer = nn.Conv2d(256, 512, 3, 2, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(512, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' 8 residual repeats '''
        layer1a = nn.Conv2d(512, 256, 1, bias = self.usebias, padding=0)
        layer1b = nn.BatchNorm2d(256, momentum= self.bn_momentum)
        layer1c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)

        layer2a = nn.Conv2d(256, 512, 3, bias= self.usebias, padding=1)
        layer2b = nn.BatchNorm2d(512, momentum= self.bn_momentum)
        layer2c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        
        for i in range(0,8):
          layer = nn.Sequential(layer1a,layer1b,layer1c,layer2a,layer2b,layer2c)
          layers.append(layer)
        
        ''' convs '''
        layer = nn.Conv2d(512, 1024, 3, 2, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(1024, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' 4 residual repeats '''
        layer1a = nn.Conv2d(1024, 512, 1, bias = self.usebias, padding=0)
        layer1b = nn.BatchNorm2d(512, momentum= self.bn_momentum)
        layer1c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
      
        layer2a = nn.Conv2d(512, 1024, 3, bias= self.usebias, padding=1)
        layer2b = nn.BatchNorm2d(1024, momentum= self.bn_momentum)
        layer2c = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        
        for i in range(0,4):
          layer = nn.Sequential(layer1a,layer1b,layer1c,layer2a,layer2b,layer2c)
          layers.append(layer)
        
        ''' convs '''
        layer = nn.Conv2d(1024, 512, 1, 1, bias = self.usebias, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(512, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        layer = nn.Conv2d(512, 1024, 3, 1, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(1024, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' scale prediction '''
        layer = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=1, padding=0),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1)
        )
        layers.append(layer)

        layer = nn.Conv2d(1024, 512, kernel_size=1, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(512)
        layers.append(layer)
        layer = nn.LeakyReLU(0.1)
        layers.append(layer)

        layer = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size= 3, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 24, kernel_size= 1)
        )
        layers.append(layer)

        ''' convs '''
        layer = nn.Conv2d(512, 256, 1, 1, bias = self.usebias, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(256, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' upsampling '''
        layer = nn.Upsample(scale_factor= 2)
        layers.append(layer,)

        ''' convs '''
        layer = nn.Conv2d(768, 256, 1, 1, bias = self.usebias, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(256, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        layer = nn.Conv2d(256, 512, 3, 1, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(512, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' scale prediction '''
        layer = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1, padding=0),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1)
        )
        layers.append(layer)

        layer = nn.Conv2d(512, 256, kernel_size=1, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(256)
        layers.append(layer)
        layer = nn.LeakyReLU(0.1)
        layers.append(layer)

        layer = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size= 3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 24, kernel_size= 1)
        )
        layers.append(layer)

        ''' convs '''
        layer = nn.Conv2d(256, 128, 1, 1, bias = self.usebias, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(128, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' upsampling '''
        layer = nn.Upsample(scale_factor= 2)
        layers.append(layer)

        ''' convs '''
        layer = nn.Conv2d(384, 128, 1, 1, bias = self.usebias, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(128, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        layer = nn.Conv2d(128, 256, 3, 1, bias = self.usebias, padding=1)
        layers.append(layer)
        layer = nn.BatchNorm2d(256, momentum= self.bn_momentum)
        layers.append(layer)
        layer = nn.LeakyReLU(negative_slope= self.leakyReLuAlpha)
        layers.append(layer)

        ''' scale prediction '''
        layer = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=1, padding=0),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1)
        )
        layers.append(layer)

        layer = nn.Conv2d(256, 128, kernel_size=1, padding=0)
        layers.append(layer)
        layer = nn.BatchNorm2d(128)
        layers.append(layer)
        layer = nn.LeakyReLU(0.1)
        layers.append(layer)

        layer = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size= 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 24, kernel_size= 1)
        )
        layers.append(layer)
        return layers

    def evaluateModel(self, inputtensor):
        route = []
        output = []
        x = self.layers[0](inputtensor)
        x = self.layers[1](x)
        x = self.layers[2](x)
        
        x = self.layers[3](x)
        x = self.layers[4](x)
        x = self.layers[5](x)
        
        x = x + self.layers[6](x)
        
        x = self.layers[7](x)
        x = self.layers[8](x)
        x = self.layers[9](x)

        x = x + self.layers[10](x)
        x = x + self.layers[11](x)
        
        x = self.layers[12](x)
        x = self.layers[13](x)
        x = self.layers[14](x)
        
        x = x + self.layers[15](x)
        x = x + self.layers[16](x)
        x = x + self.layers[17](x)
        x = x + self.layers[18](x)
        x = x + self.layers[19](x)
        x = x + self.layers[20](x)
        x = x + self.layers[21](x)
        x = x + self.layers[22](x)
        route.append(x)
        
        x = self.layers[23](x)
        x = self.layers[24](x)
        x = self.layers[25](x)
        
        x = x + self.layers[26](x)
        x = x + self.layers[27](x)
        x = x + self.layers[28](x)
        x = x + self.layers[29](x)
        x = x + self.layers[30](x)
        x = x + self.layers[31](x)
        x = x + self.layers[32](x)
        x = x + self.layers[33](x)
        route.append(x)
        
        x = self.layers[34](x)
        x = self.layers[35](x)
        x = self.layers[36](x)
        
        x = x + self.layers[37](x)
        x = x + self.layers[38](x)
        x = x + self.layers[39](x)
        x = x + self.layers[40](x)

        x = self.layers[41](x)
        x = self.layers[42](x)
        x = self.layers[43](x)
        
        x = self.layers[44](x)
        x = self.layers[45](x)
        x = self.layers[46](x)

        x = self.layers[47](x)
        x = self.layers[48](x)
        x = self.layers[49](x)
        xx = self.layers[50](x)
        x = self.layers[51](xx)
        x = x.reshape(xx.shape[0], 3, 8, xx.shape[2], xx.shape[3]).permute(0,1,3,4,2)
        output.append(x)

        x = self.layers[52](xx)
        x = self.layers[53](x)
        x = self.layers[54](x)
        
        x = self.layers[55](x)
        x = torch.cat([x, route[-1]], dim=1)
        route.pop()

        x = self.layers[56](x)
        x = self.layers[57](x)
        x = self.layers[58](x)
        
        x = self.layers[59](x)
        x = self.layers[60](x)
        x = self.layers[61](x)
        
        x = self.layers[62](x)
        x = self.layers[63](x)
        x = self.layers[64](x)
        xx = self.layers[65](x)
        x = self.layers[66](xx)
        x = x.reshape(xx.shape[0], 3, 8, xx.shape[2], xx.shape[3]).permute(0,1,3,4,2)
        output.append(x)

        x = self.layers[67](xx)
        x = self.layers[68](x)
        x = self.layers[69](x)
        
        x = self.layers[70](x)
        x = torch.cat([x, route[-1]], dim=1)
        route.pop()

        x = self.layers[71](x)
        x = self.layers[72](x)
        x = self.layers[73](x)
        
        x = self.layers[74](x)
        x = self.layers[75](x)
        x = self.layers[76](x)

        x = self.layers[77](x)
        x = self.layers[78](x)
        x = self.layers[79](x)
        xx = self.layers[80](x)
        x = self.layers[81](xx)
        x = x.reshape(xx.shape[0], 3, 8, xx.shape[2], xx.shape[3]).permute(0,1,3,4,2)
        output.append(x)

        return output


In [41]:
num_classes = 3
IMAGE_SIZE = 224
model = yolo_model()
out = model.evaluateModel(x)

In [50]:
model.layers

ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.1)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.1)
  (6): Sequential(
    (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.999, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1)
  )
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (8): BatchNorm2d(128, eps=1e-05, momen

In [47]:
out[0].shape

torch.Size([1, 3, 7, 7, 8])

In [48]:
out[1].shape

torch.Size([1, 3, 14, 14, 8])

In [49]:
out[2].shape

torch.Size([1, 3, 28, 28, 8])